In [22]:
%load_ext autoreload
%autoreload 2

import qubx
%qubxd

from pathlib import Path
from qubx.core.lookups import lookup
from qubx.core.basics import DataType
from qubx import logger, QubxLogConfig
from qubx.utils.runner.runner import run_strategy, StrategyConfig, AccountConfigurationManager, ExchangeConfig, LoggingConfig
from qubx.core.interfaces import IStrategy, IStrategyContext, BaseErrorEvent
from qubx.connectors.ccxt.utils import instrument_to_ccxt_symbol, ccxt_convert_orderbook

QubxLogConfig.set_log_level("INFO")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
instr = lookup.find_symbol("BITMEX", "BTCUSD")
assert instr is not None, "Got wrong instrument"
instr

BITMEX:SWAP:BTCUSD

In [42]:
instrument_to_ccxt_symbol(instr)

'BTC/USD:BTC'

In [41]:
instr.exchange_symbol

'XBTUSD'

In [55]:
class TestStrategy(IStrategy):
    def on_error(self, ctx: IStrategyContext, error: BaseErrorEvent) -> None:
        logger.error(f"Error: {error}")

    def on_stop(self, ctx: IStrategyContext):
        logger.info("Stopping test strategy")


ctx = run_strategy(
    config=StrategyConfig(
        name="TestStrategy",
        strategy=TestStrategy,
        exchanges={
            "BINANCE.UM": ExchangeConfig(
                connector="ccxt",
                universe=["BTCUSDT"],
            ),
            "BITMEX": ExchangeConfig(
                connector="ccxt",
                universe=["BTCUSD"],
            ),
            "KRAKEN.F": ExchangeConfig(
                connector="ccxt",
                universe=["BTCUSD"],
            ),
        },
        logging=LoggingConfig(
            logger="InMemoryLogsWriter",
            position_interval="10s",
            portfolio_interval="1m",
            heartbeat_interval="10m",
        )
    ),
    account_manager=AccountConfigurationManager(),
    paper=True,
    blocking=False,
)

2025-04-03 13:36:59.202 [ℹ️] BINANCE.UM Initialized
2025-04-03 13:36:59.208 [ℹ️] BITMEX Initialized
2025-04-03 13:36:59.213 [ℹ️] KRAKEN.F Initialized
2025-04-03 13:36:59.213 [ℹ️] - Strategy: TestStrategy
- Mode: paper
- Parameters: {}
2025-04-03 13:36:59.217 [ℹ️] KRAKEN.F Listening to BTC/USD:USD orderbook
2025-04-03 13:36:59.218 [ℹ️] BINANCE.UM Listening to BTC/USDT:USDT orderbook
2025-04-03 13:36:59.218 [ℹ️] [StrategyContext] :: Start processing market data
2025-04-03 13:36:59.218 [ℹ️] BITMEX Listening to BTC/USD:BTC orderbook
2025-04-03 13:36:59.219 [ℹ️] [StrategyContext] :: strategy is started in thread


2025-04-03 13:36:59.758 [ℹ️] Heartbeat at 2025-04-03T13:30:00
2025-04-03 13:37:47.310 [ℹ️] BITMEX Listening to BTC/USD:BTC trade
2025-04-03 13:37:47.312 [ℹ️] KRAKEN.F Listening to BTC/USD:USD trade
2025-04-03 13:37:47.313 [ℹ️] BINANCE.UM Listening to BTC/USDT:USDT trade
2025-04-03 13:40:00.006 [ℹ️] Heartbeat at 2025-04-03T13:40:00
2025-04-03 13:43:57.162 [ℹ️] Stopping test strategy
2025-04-03 13:43:57.182 [ℹ️] BINANCE.UM BTC/USDT:USDT orderbook listening has been stopped
2025-04-03 13:43:57.194 [ℹ️] BINANCE.UM BTC/USDT:USDT trade listening has been stopped
2025-04-03 13:43:57.441 [ℹ️] BITMEX BTC/USD:BTC trade listening has been stopped
2025-04-03 13:43:57.443 [ℹ️] BITMEX BTC/USD:BTC orderbook listening has been stopped
2025-04-03 13:43:57.693 [ℹ️] KRAKEN.F BTC/USD:USD trade listening has been stopped
2025-04-03 13:43:57.694 [ℹ️] KRAKEN.F BTC/USD:USD orderbook listening has been stopped
2025-04-03 13:43:57.944 [ℹ️] [StrategyContext] :: Market data processing stopped


In [47]:
ctx.instruments

[BINANCE.UM:SWAP:BTCUSDT, BITMEX:SWAP:BTCUSD]

In [77]:
instr1 = ctx.instruments[0]
instr2 = ctx.instruments[1]
instr3 = ctx.instruments[2]

q1 = ctx.quote(instr1)
q2 = ctx.quote(instr2)
q3 = ctx.quote(instr3)

print(f"{instr1}: {q1}")
print(f"{instr2}: {q2}")
print(f"{instr3}: {q3}")


BINANCE.UM:SWAP:BTCUSDT: [2025-04-03T13:43:49.443000000]	82265.90000 (9.0) | 82266.00000 (20.6)
BITMEX:SWAP:BTCUSD: [2025-04-03T13:43:49.545000000]	82245.00000 (100.0) | 82245.10000 (98400.0)
KRAKEN.F:SWAP:BTCUSD: [2025-04-03T13:43:49.575000000]	82287.00000 (0.1) | 82288.00000 (1.3)


In [62]:
ctx.get_balances(exchange="KRAKEN.F")

{'USDT': AssetBalance(free=100000.0, locked=0.0, total=100000.0)}

In [74]:
ctx.trade(instr3, 1.0)

Order(id='SIM-ORDER-BTCUSD-100004', type='MARKET', instrument=KRAKEN.F:SWAP:BTCUSD, time=numpy.datetime64('2025-04-03T13:42:42.282391296'), quantity=1.0, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='qubx_BTCUSD_17436875922', cost=0.0, options={})

In [75]:
ctx.get_leverages(exchange="KRAKEN.F")

{KRAKEN.F:SWAP:BTCUSD: 0.0}

In [76]:
ctx.positions

{BINANCE.UM:SWAP:BTCUSDT: 2025-04-03 13:42:46 [BINANCE.UM:SWAP:BTCUSDT] qty=0.000 entryPrice=0.0 price=82340.1 PNL: (unrealized=0.00 realized=0.00 pnl=0.00) value=0.00,
 BITMEX:SWAP:BTCUSD: 2025-04-03 13:42:46 [BITMEX:SWAP:BTCUSD] qty=0.00 entryPrice=0.0 price=82312.7 PNL: (unrealized=0.00 realized=0.00 pnl=0.00) value=0.00,
 KRAKEN.F:SWAP:BTCUSD: 2025-04-03 13:42:46 [KRAKEN.F:SWAP:BTCUSD] qty=0.0000 entryPrice=0 price=82352 PNL: (unrealized=0.00 realized=-308.00 pnl=-308.00) value=0.00}

In [58]:
ctx.subscribe(DataType.TRADE)

In [59]:
ctx.get_data(instr3, DataType.TRADE)[-10:]

[[2025-04-03T13:37:51.151000000]	82320.00000 (0.06) buy ,
 [2025-04-03T13:37:51.183000000]	82320.00000 (0.07) buy ,
 [2025-04-03T13:37:51.193000000]	82320.00000 (0.06) buy ,
 [2025-04-03T13:37:54.408000000]	82304.00000 (0.06) sell ,
 [2025-04-03T13:37:54.427000000]	82299.00000 (0.06) sell ,
 [2025-04-03T13:37:54.929000000]	82289.00000 (0.04) sell ,
 [2025-04-03T13:37:54.984000000]	82278.00000 (0.01) sell ,
 [2025-04-03T13:37:55.061000000]	82275.00000 (0.01) sell ,
 [2025-04-03T13:37:55.061000000]	82275.00000 (0.01) sell ,
 [2025-04-03T13:37:55.235000000]	82257.00000 (0.01) sell ]

In [78]:
ctx.stop()

In [17]:
import ccxt.pro as cxp

bitmex = cxp.bitmex()
await bitmex.load_markets();

In [38]:
ob = await bitmex.watch_order_book_for_symbols(["BTC/USDT:USDT"])

In [44]:
bitmex.market(instrument_to_ccxt_symbol(instr))

{'id': 'XBTUSD',
 'lowercaseId': None,
 'symbol': 'BTC/USD:BTC',
 'base': 'BTC',
 'quote': 'USD',
 'settle': 'BTC',
 'baseId': 'XBT',
 'quoteId': 'USD',
 'settleId': 'XBt',
 'type': 'swap',
 'spot': False,
 'margin': False,
 'swap': True,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': True,
 'linear': False,
 'inverse': True,
 'subType': 'inverse',
 'taker': 0.0005,
 'maker': 0.0005,
 'contractSize': 100000000.0,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 100.0,
  'price': 0.1,
  'cost': None,
  'base': None,
  'quote': None},
 'limits': {'leverage': {'min': 1.0, 'max': 100.0},
  'amount': {'min': None, 'max': None},
  'price': {'min': None, 'max': 1000000.0},
  'cost': {'min': None, 'max': 10000000.0}},
 'marginModes': {'cross': None, 'isolated': None},
 'created': 1463140800000,
 'info': {'symbol': 'XBTUSD',
  'rootSymbol': 'XBT',
  'state': 'Open',
  'typ': 'FFWCSX',
  'listing': '2016-0

In [42]:
await bitmex.close()

In [18]:
try:
    while True:
        trades = await bitmex.watch_trades_for_symbols(["BTC/USDT:USDT"])
        for trade in trades:
            print(trade)
except KeyboardInterrupt:
    pass

{'info': {'timestamp': '2025-04-03T13:25:24.568Z', 'symbol': 'XBTUSDT', 'side': 'Buy', 'size': 58500, 'price': 81949.9, 'tickDirection': 'PlusTick', 'trdMatchID': '00000000-006d-1000-0000-0015cc7cedb2', 'grossValue': 4794069150, 'homeNotional': 0.0585, 'foreignNotional': 4794.06915, 'trdType': 'Regular'}, 'timestamp': 1743686724568, 'datetime': '2025-04-03T13:25:24.568Z', 'symbol': 'BTC/USDT:USDT', 'id': '00000000-006d-1000-0000-0015cc7cedb2', 'order': None, 'type': None, 'takerOrMaker': None, 'side': 'buy', 'price': 81949.9, 'cost': 4794.06915, 'amount': 58500.0, 'fee': {'cost': None, 'currency': None}, 'fees': []}
{'info': {'timestamp': '2025-04-03T13:25:36.051Z', 'symbol': 'XBTUSDT', 'side': 'Buy', 'size': 400, 'price': 81942.0, 'tickDirection': 'MinusTick', 'trdMatchID': '00000000-006d-1000-0000-0015cc7dae27', 'grossValue': 32776800, 'homeNotional': 0.0004, 'foreignNotional': 32.7768, 'trdType': 'Regular'}, 'timestamp': 1743686736051, 'datetime': '2025-04-03T13:25:36.051Z', 'symbol

CancelledError: 

In [64]:
from qubx.connectors.ccxt.utils import ccxt_convert_trade
ccxt_convert_trade(trades[0])

[2025-04-03T13:20:38.069000000]	81863.70000 (10000.00) buy 